# Multilayer perceptron

### **1. Layers and Weights**
For a neural network with L layers, let:
- $x \in \mathbb{R}^d$ be the input vector (of dimension d),
- $W^{[l]} \in \mathbb{R}^{n_l \times n_{l-1}}$ be the weight matrix for the l-th layer,
- $b^{[l]} \in \mathbb{R}^{n_l}$ be the bias vector for the l-th layer,
- $a^{[l]} \in \mathbb{R}^{n_l}$ be the activation vector for the l-th layer.

The output of each layer is computed as:
$$ z^{[l]} = W^{[l]} a^{[l-1]} + b^{[l]} $$
where $a^{[l-1]}$ is the activation vector from the previous layer, and $z^{[l]}$ is the linear combination of inputs.

### **2. Activation Functions**
After computing the linear combination, the output is passed through an activation function $f(z)$:
- **Sigmoid activation function**: 
  $$ \sigma(z) = \frac{1}{1 + e^{-z}} $$  
- **ReLU activation function**: 
  $$ \text{ReLU}(z) = \max(0, z) $$  

For the output layer:
- If the task is classification, the output could be the **softmax** or a simple **sigmoid** for binary classification.

### **3. Forward Pass**
The forward pass involves computing the activations at each layer until the final output:
$$ a^{[l]} = f(z^{[l]}) $$

### **4. Backpropagation**
During training, **backpropagation** is used to adjust the weights. The gradients of the loss function L with respect to the weights are computed:
$$ \frac{\partial L}{\partial W^{[l]}} = \frac{\partial L}{\partial a^{[l]}} \frac{\partial a^{[l]}}{\partial W^{[l]}} $$


In [1]:
import numpy as np

class MLP:
    def __init__(self, layer_sizes, activation_function='sigmoid'):
        """
        layer_sizes: List containing the number of neurons in each layer.
        activation_function: 'sigmoid' or 'relu'.
        """
        self.layer_sizes = layer_sizes
        self.activation_function = activation_function
        
        # Initialize weights and biases
        self.weights = [np.random.randn(next_layer, current_layer) * 0.1 for current_layer, next_layer in zip(layer_sizes[:-1], layer_sizes[1:])]
        self.biases = [np.zeros((size, 1)) for size in layer_sizes[1:]]
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def sigmoid_derivative(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))
    
    def relu(self, z):
        return np.maximum(0, z)
    
    def relu_derivative(self, z):
        return np.where(z > 0, 1, 0)
    
    def forward(self, X):
        """Forward pass through the network"""
        self.a = [X]  # Store activations for each layer
        self.z = []  # Store linear combinations for each layer
        
        for i in range(len(self.layer_sizes) - 1):
            z = np.dot(self.weights[i], self.a[-1]) + self.biases[i]
            self.z.append(z)
            if self.activation_function == 'sigmoid':
                a = self.sigmoid(z)
            elif self.activation_function == 'relu':
                a = self.relu(z)
            self.a.append(a)
        
        return self.a[-1]
    
    def backward(self, X, y, learning_rate=0.01):
        """Backward pass to update weights and biases"""
        m = X.shape[1]
        self.dz = [self.a[-1] - y]  # Derivative of the cost function wrt to output
        self.dw = [np.dot(self.dz[0], self.a[-2].T) / m]
        self.db = [np.sum(self.dz[0], axis=1, keepdims=True) / m]

        for l in range(len(self.layer_sizes) - 2, -1, -1):
            if self.activation_function == 'sigmoid':
                dz = self.dz[-1] * self.sigmoid_derivative(self.z[l])
            elif self.activation_function == 'relu':
                dz = self.dz[-1] * self.relu_derivative(self.z[l])
            self.dz.append(dz)
            
            dw = np.dot(dz, self.a[l].T) / m
            db = np.sum(dz, axis=1, keepdims=True) / m
            self.dw.append(dw)
            self.db.append(db)
        
        # Update weights and biases
        self.weights = [w - learning_rate * dw for w, dw in zip(self.weights, reversed(self.dw))]
        self.biases = [b - learning_rate * db for b, db in zip(self.biases, reversed(self.db))]

    def train(self, X, y, epochs=1000, learning_rate=0.01):
        """Train the MLP using forward and backward propagation"""
        for epoch in range(epochs):
            # Forward pass
            self.forward(X)
            # Backward pass
            self.backward(X, y, learning_rate)
            if epoch % 100 == 0:
                loss = np.mean(np.square(y - self.a[-1]))  # Mean squared error
                print(f"Epoch {epoch}: Loss = {loss}")

    def predict(self, X):
        output = self.forward(X)
        return output


In [2]:
if __name__ == "__main__":
    # XOR problem
    X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]).T 
    y = np.array([[0], [1], [1], [0]]).T 

    # Initialize the MLP with 2 input neurons, 2 hidden neurons, and 1 output neuron
    mlp = MLP(layer_sizes=[2, 2, 1], activation_function='sigmoid')

    mlp.train(X, y, epochs=10000, learning_rate=0.1)

    predictions = mlp.predict(X)
    print("Predictions:")
    print(predictions)

Epoch 0: Loss = 0.2508933402077232
Epoch 100: Loss = 0.2501496210621299
Epoch 200: Loss = 0.2500240294205483
Epoch 300: Loss = 0.2500032297799143
Epoch 400: Loss = 0.24999982488751438
Epoch 500: Loss = 0.2499992860049544
Epoch 600: Loss = 0.24999921751241574
Epoch 700: Loss = 0.24999922570234906
Epoch 800: Loss = 0.24999924597992446
Epoch 900: Loss = 0.2499992677667775
Epoch 1000: Loss = 0.24999928934283983
Epoch 1100: Loss = 0.2499993104400933
Epoch 1200: Loss = 0.2499993310282968
Epoch 1300: Loss = 0.24999935111583474
Epoch 1400: Loss = 0.24999937071700779
Epoch 1500: Loss = 0.2499993898466479
Epoch 1600: Loss = 0.24999940851923538
Epoch 1700: Loss = 0.24999942674876516
Epoch 1800: Loss = 0.24999944454873932
Epoch 1900: Loss = 0.2499994619321814
Epoch 2000: Loss = 0.2499994789116549
Epoch 2100: Loss = 0.2499994954992814
Epoch 2200: Loss = 0.24999951170675858
Epoch 2300: Loss = 0.24999952754537702
Epoch 2400: Loss = 0.24999954302603677
Epoch 2500: Loss = 0.24999955815926253
Epoch 2600